---
### **Depthwise Separable Convolution**  
---
![DWConv](DWConv/dwconv.png)
---
> #### Theory
>> - **Depthwise Separable Convolution은 연산량을 줄이지만 성능은 유지하는 일종의 알고리즘이다.**  
>> - **Depthwise Separable Convolution은 Depthwise Convolution과 Pointwise COnvolution의 결합이다.**
>> - **EX) 3 X 3 X 3 입력에 대하여 Conv 연산후  3 X 3 X 3 출력을 하는 경우.**  
>>> - **1) 일반적인 Convolution 방법**  
>>> **Origin Image = 3 X 3 X 3 (H, W, C)**  
>>> **아웃풋의 채널이 3이므로 커널의 크기는 3이고 이에 따라**  
>>> **Origin Image x Size of Kernel = 3 X 3 X 3 X 3 = 81**  
>>> **Result to Calculate Conv param = 81**   
---
>>> - **(2) DWConv의 방법**   
>>> **1. Origin Image = 3 X 3 X 3 (H, W, C)**    
>>> **1-1. 아웃풋의 채널이 3이므로 커널의 크기는 3이고 이에 따라**    
>>> **::Depthwise Separable Conv: 3 x 3 x 1 x 3 + 1 X 1 X 3 X 3 = 27 + 9 = 36**    
>>> **>>Depthwise Kernel = 3 x 3 x 1 x 3 (H, W, C, num of Kernel)**  
>>> **>>Pointwise Kernel = 1 x 1 x 3 x 3 (H, W, C, num of Kernel)**  
---
>>> - **(3) DWConv 과정**  
>>> **1. 입력 이미지의 Channel의 수에 맞는 Depthwise Conv를 선진행한다.**  
>>> **2. Depthwise의 Kernel 개수는 입력 이미지의 Channel의 수와 동일하다.**  
>>> **3. 단, Depthwise의 Kernel 차원은 1차원이다.**  
>>> **4. Depthwise Conv 연산 이후 입력 이미지의 차원을 다시 합쳐 하나의 영상으로 재조합한다.**  
>>> **5. 이후 얻게되는 입력 이미지의 H,W,C 데이터에 Pointwise연산을 진행한다.**    
>>> **6. Pointwise Conv에 사용되는 Kernel의 사이즈는 1 x 1 이고 개수는 입력이미지의 채널과 동일하다.**  
>>> **7. 입력이미지가 3 x 3 x 3 (H,W,C)일 때 Pointwise는 1 x 1 x 3이 된다.**    

---
> #### Example Test on MobileNet  
>> - **MobileNet Architecture**  
>> - **MobileNet은 VGG와 구조적으로 비슷하다. 기존의 Conv을 DWConv로 대체하고**  
>> - **Pooling 대신에 Stride '2' 를 사용하여 Size를 축소한것이 큰 차이이다.**  
>> - **연산량은 큰폭으로 줄이고 성능은 비슷한 수준으로 유지하였다.**  

![mobilenet_architecture1](DWConv/mobilenet_architecture1.png)  
![mobilenet_architecture2](DWConv/mobilenet_architecture.png)  

---
> MobileNet_VER1 Import Lib

In [1]:
import torch
import torch.nn as nn
from torchsummary import summary

/home/park/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---
> MobileNet_VER1 Class

In [2]:
class MobileNet(nn.Module):
    def __init__(self, in_ch = 3, out_ch = 1, cls = 1000): 
        #input_image = 224 x 224 x 3 (H,W,C)
        #output_image = 1 x 1 x 1000 (H,W,C)
        super().__init__()
        self.DWConv1 = nn.Sequential(
            nn.Conv2d(in_channels=in_ch, out_channels=32, kernel_size=3, stride= 2, padding=1, bias=False),
            nn.Conv2d(in_channels= 32, out_channels= 32,  kernel_size= 3, stride = 1, padding = 1, groups=32, bias=False), # Depthwise
            nn.Conv2d(in_channels= 32, out_channels= 64,  kernel_size= 1, stride = 1, padding = 0, bias=False), #Pointwise
        )
        self.DWConv2 = nn.Sequential(
            nn.Conv2d(in_channels= 64, out_channels= 64,  kernel_size= 3, stride = 2, padding = 1, groups=64, bias=False), # Depthwise
            nn.Conv2d(in_channels= 64, out_channels= 128,  kernel_size= 1, stride = 1, padding = 0, bias=False), #Pointwise
        )
        self.DWConv3 = nn.Sequential(
            nn.Conv2d(in_channels= 128, out_channels= 128,  kernel_size= 3, stride = 1, padding = 1, groups=128, bias=False), # Depthwise
            nn.Conv2d(in_channels= 128, out_channels= 128,  kernel_size= 1, stride = 1, padding = 0, bias=False), #Pointwise
            nn.Conv2d(in_channels= 128, out_channels= 128,  kernel_size= 3, stride = 2, padding = 1, groups=128, bias=False), # Depthwise
            nn.Conv2d(in_channels= 128, out_channels= 256,  kernel_size= 1, stride = 1, padding = 0, bias=False), #Pointwise
        )
        self.DWConv4 = nn.Sequential(
            nn.Conv2d(in_channels= 256, out_channels= 256,  kernel_size= 3, stride = 1, padding = 1, groups=256, bias=False), # Depthwise
            nn.Conv2d(in_channels= 256, out_channels= 256,  kernel_size= 1, stride = 1, padding = 0, bias=False), #Pointwise
            nn.Conv2d(in_channels= 256, out_channels= 256,  kernel_size= 3, stride = 2, padding = 1, groups=256, bias=False), # Depthwise
            nn.Conv2d(in_channels= 256, out_channels= 512,  kernel_size= 1, stride = 1, padding = 0, bias=False), #Pointwise
        )
        self.DWConv5 = nn.Sequential(
            nn.Conv2d(in_channels= 512, out_channels= 512,  kernel_size= 3, stride = 1, padding = 1, groups=512, bias=False), # Depthwise
            nn.Conv2d(in_channels= 512, out_channels= 512,  kernel_size= 1, stride = 1, padding = 0, bias=False), #Pointwise
        )
        self.DWConv6 = nn.Sequential(
            nn.Conv2d(in_channels= 512, out_channels= 512,  kernel_size= 3, stride = 2, padding = 1, groups=512, bias=False), # Depthwise
            nn.Conv2d(in_channels= 512, out_channels= 1024,  kernel_size= 1, stride = 1, padding = 0, bias=False), #Pointwise
        )
        self.DWConv7 = nn.Sequential(
            nn.Conv2d(in_channels= 1024, out_channels= 1024,  kernel_size= 3, stride = 2, padding = 1, groups=1024, bias=False), # Depthwise
            nn.Conv2d(in_channels= 1024, out_channels= 1024,  kernel_size= 1, stride = 1, padding = 0, bias=False), #Pointwise
        )
        self.avgp = nn.AvgPool2d(kernel_size=1, stride=1)
        self.fc = nn.Linear(in_features=1024, out_features=cls)
        self.sm = nn.Softmax()
    
    def forward(self,x):
        out = self.DWConv1(x)
        out = self.DWConv2(out)
        out = self.DWConv3(out)
        out = self.DWConv4(out)
        out = self.DWConv5(out)
        out = self.DWConv5(out)
        out = self.DWConv5(out)
        out = self.DWConv5(out)
        out = self.DWConv5(out)
        out = self.DWConv6(out)
        out = self.DWConv7(out)
        return out


---
> MobileNet_VER1 Main

In [3]:
device = torch.device('cpu')

if __name__ == "__main__":
    model = MobileNet()
    summary(model=model, input_size= (3,224,224), device= device.type)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
            Conv2d-2         [-1, 32, 112, 112]             288
            Conv2d-3         [-1, 64, 112, 112]           2,048
            Conv2d-4           [-1, 64, 56, 56]             576
            Conv2d-5          [-1, 128, 56, 56]           8,192
            Conv2d-6          [-1, 128, 56, 56]           1,152
            Conv2d-7          [-1, 128, 56, 56]          16,384
            Conv2d-8          [-1, 128, 28, 28]           1,152
            Conv2d-9          [-1, 256, 28, 28]          32,768
           Conv2d-10          [-1, 256, 28, 28]           2,304
           Conv2d-11          [-1, 256, 28, 28]          65,536
           Conv2d-12          [-1, 256, 14, 14]           2,304
           Conv2d-13          [-1, 512, 14, 14]         131,072
           Conv2d-14          [-1, 512,

---
> MobileNet_VER1 Result

![ver1_result](DWConv/result1.png)

---
> MobileNet_VER2 Import Lib

In [4]:
import torch
import torch.nn as nn
from torchsummary import summary

---
> MobileNet_VER2 Class

In [5]:
class MobileNet(nn.Module):
    def __init__(self): 
        super(MobileNet, self).__init__()
        def InitConv():
            return nn.Sequential(
                nn.Conv2d(3, 32, 3, 2, 1, bias=False),
                nn.BatchNorm2d(32),
                nn.ReLU(inplace=True),            
            )    

        def DWConv(in_ch, out_ch, dw_stride):
            return nn.Sequential(
                #dw
                nn.Conv2d(in_ch, in_ch, 3, dw_stride, padding=1, groups=in_ch, bias=False),
                nn.BatchNorm2d(in_ch),
                nn.ReLU(inplace=True),

                #pw
                nn.Conv2d(in_ch, out_ch, 1, 1, bias=False),
                nn.BatchNorm2d(out_ch),
                nn.ReLU(inplace=True),            
            )
    
        self.model = nn.Sequential(
            InitConv(),
            DWConv(32, 64, 1),
            DWConv(64, 128, 2),
            DWConv(128, 128, 1),
            DWConv(128, 256, 2),
            DWConv(256, 256, 1),
            DWConv(256, 512, 2),
            DWConv(512, 512, 1),
            DWConv(512, 512, 1),
            DWConv(512, 512, 1),
            DWConv(512, 512, 1),
            DWConv(512, 512, 1),
            DWConv(512, 1024, 2),
            DWConv(1024, 1024, 1),
            nn.AvgPool2d(1)        
        )
        self.fc = nn.Linear(1024, 1000)
    
    def forward(self,x):
        out = self.model(x)  
        out = out.view(-1, 1024)
        out = self.fc(out)      
        return out

---
> MobileNet_VER2 Main

In [6]:
device = torch.device('cpu')

if __name__ == "__main__":
    model = MobileNet()
    summary(model=model, input_size= (3,224,224), device= device.type)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              ReLU-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
              ReLU-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 64, 112, 112]           2,048
       BatchNorm2d-8         [-1, 64, 112, 112]             128
              ReLU-9         [-1, 64, 112, 112]               0
           Conv2d-10           [-1, 64, 56, 56]             576
      BatchNorm2d-11           [-1, 64, 56, 56]             128
             ReLU-12           [-1, 64, 56, 56]               0
           Conv2d-13          [-1, 128, 56, 56]           8,192
      BatchNorm2d-14          [-1, 128,

---
> MobileNet_VER2 Result

![ver2_result](DWConv/result2.png)